In [1]:
# !pip install pandas tqdm transformers accelerate bitsandbytes

In [2]:
import re
import os
import pandas as pd
from tqdm import tqdm

import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [3]:
test = pd.read_csv('../data/test.csv')
test

,ID,Question
0,TEST_000,금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...
1,TEST_001,위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...
2,TEST_002,관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n...
3,TEST_003,재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?\n1 복구 절차 수립...
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...
...,...,...
510,TEST_510,"""정보보호최고책임자""의 임명에 관한 설명으로 옳지 않은 것은?\n1 정보보호최고책임..."
511,TEST_511,IPv6 주소 체계의 주요 특징으로 옳지 않은 것은?\n1 NAT 필요성 감소\n2...
512,TEST_512,하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는...
513,TEST_513,전자금융거래법의 주요 목적 중 하나는 무엇인가?\n1 전자금융거래의 비대면성 강화\...


In [4]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2


# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())

    question = " ".join(q_lines)
    return question, options

In [5]:
# 프롬프트 생성기
def make_prompt_auto(text):
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                # "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                "아래 질문에 대해 정답의 핵심 키워드와 의미를 모두 포함하여 3문장 이내로 간결하게 답변하세요. 군더더기 없이 요점만 명확하게 작성하세요.\n\n"
                f"질문: {text}\n\n"
                "답변:"
                )
    return prompt

In [6]:
# model_name = "nlpai-lab/KULLM3"
# model_name = "KRX-Data/WON-Reasoning"
# model_name = "beomi/KoAlpaca-Polyglot-12.8B"
# model_name = "nlpai-lab/kullm-polyglot-5.8b-v2"
model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

# Tokenizer 및 모델 로드 (4bit)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

# Inference pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [7]:
# 후처리 함수
def extract_answer_only(generated_text: str, original_question: str) -> str:
    """
    - "답변:" 이후 텍스트만 추출
    - 객관식 문제면: 정답 숫자만 추출 (실패 시 전체 텍스트 또는 기본값 반환)
    - 주관식 문제면: 전체 텍스트 그대로 반환
    - 공백 또는 빈 응답 방지: 최소 "미응답" 반환
    """
    # "답변:" 기준으로 텍스트 분리
    if "답변:" in generated_text:
        text = generated_text.split("답변:")[-1].strip()
    else:
        text = generated_text.strip()

    # 공백 또는 빈 문자열일 경우 기본값 지정
    if not text:
        return "미응답"

    # 객관식 여부 판단
    is_mc = is_multiple_choice(original_question)

    if is_mc:
        # 숫자만 추출
        match = re.match(r"\D*([1-9][0-9]?)", text)
        if match:
            return match.group(1)
        else:
            # 숫자 추출 실패 시 "0" 반환
            return "0"
    else:
        return text


In [8]:
preds = []

for q in tqdm(test['Question'], desc="Inference"):
    print("#################### Question ###########################")
    prompt = make_prompt_auto(q)
    print(prompt)
    output = pipe(prompt, max_new_tokens=256, temperature=0.2, top_p=0.9)
    pred_answer = extract_answer_only(output[0]["generated_text"], original_question=q)
    print("#################### Answer ###########################")
    print(pred_answer)
    preds.append(pred_answer)

Inference:   0%|          | 0/515 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


#################### Question ###########################
당신은 금융보안 전문가입니다.
아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.

질문: 금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
선택지:
1 소비자금융업
2 투자자문업
3 투자매매업
4 투자중개업
5 보험중개업

답변:


Inference:   0%|          | 1/515 [00:11<1:41:55, 11.90s/it]

#################### Answer ###########################
2
#################### Question ###########################
당신은 금융보안 전문가입니다.
아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.

질문: 위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?
선택지:
1 수행인력
2 위험 수용
3 위험 대응 전략 선정
4 대상
5 기간

답변:


Inference:   0%|          | 2/515 [00:23<1:40:07, 11.71s/it]

...

#################### Answer ###########################
1
#################### Question ###########################
당신은 금융보안 전문가입니다.
아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.

질문: 개인정보보호법 제22조의2에 따라 만 14세 미만 아동의 개인정보를 처리하기 위해 필요한 절차로 옳은 것은?
선택지:
1 아동의 학교의 동의를 받아야 한다.
2 법정대리인의 동의를 받아야 한다.
3 아동 본인의 동의만 받으면 된다.
4 아동의 친구의 동의를 받아야 한다.

답변:


Inference:   1%|▏         | 7/515 [01:21<1:38:09, 11.59s/it]

#################### Answer ###########################
3
#################### Question ###########################
당신은 금융보안 전문가입니다.
아래 질문에 대해 정답의 핵심 키워드와 의미를 모두 포함하여 3문장 이내로 간결하게 답변하세요. 군더더기 없이 요점만 명확하게 작성하세요.

질문: 전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.

답변:


Inference:   1%|▏         | 7/515 [01:31<1:51:05, 13.12s/it]


KeyboardInterrupt: 

In [ ]:
sample_submission = pd.read_csv('../submission/sample_submission.csv')
sample_submission['Answer'] = preds
sample_submission.to_csv('../submission/baseline_submission.csv', index=False, encoding='utf-8-sig')